In [30]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import string

In [31]:
keggle = '/kaggle/input/spam-message-dataset/Spam/spam (1).tsv'
try:
    df = pd.read_csv('/kaggle/input/spam-message-dataset/Spam/spam (1).tsv', sep='\t', names=['Class', 'Message'])
except FileNotFoundError:
    df = pd.read_csv("E:/Datasets/Spam/spam.tsv", sep='\t', names=['Class', 'Message'])
df.head()

,Class,Message
0,ham,I've been searching for the right words to tha...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,"Nah I don't think he goes to usf, he lives aro..."
3,ham,Even my brother is not like to speak with me. ...
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!!


In [32]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 5567 entries, 0 to 5566
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   Class    5567 non-null   str  
 1   Message  5567 non-null   str  
dtypes: str(2)
memory usage: 542.2 KB


In [33]:
df['Length'] = df['Message'].apply(len)

In [34]:
df.head()

,Class,Message,Length
0,ham,I've been searching for the right words to tha...,196
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
2,ham,"Nah I don't think he goes to usf, he lives aro...",61
3,ham,Even my brother is not like to speak with me. ...,77
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!!,36


In [35]:
df.describe()

,Length
count,5567.000000
mean,80.450153
std,59.891023
min,2.000000
25%,36.000000
50%,62.000000
75%,122.000000
max,910.000000


In [36]:
df['Class'].value_counts()

Class
ham     4821
spam     746
Name: count, dtype: int64

### Text Preprocessing

In [37]:
# converting class column to int type where ham = 1 and spam = 0
df['Class'] = df['Class'].map({'ham': 1, 'spam' : 0})
df['Class'] = df['Class'].astype(int)
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 5567 entries, 0 to 5566
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   Class    5567 non-null   int64
 1   Message  5567 non-null   str  
 2   Length   5567 non-null   int64
dtypes: int64(2), str(1)
memory usage: 568.7 KB


In [38]:
df.head()

,Class,Message,Length
0,1,I've been searching for the right words to tha...,196
1,0,Free entry in 2 a wkly comp to win FA Cup fina...,155
2,1,"Nah I don't think he goes to usf, he lives aro...",61
3,1,Even my brother is not like to speak with me. ...,77
4,1,I HAVE A DATE ON SUNDAY WITH WILL!!!,36


### Removing Punctuation

In [39]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [40]:
# creating the function for removing puntuation
def remove_punctuation(text):
    text = ''.join([char for char in text if char not in string.punctuation])
    return text

# test the function
s = 'data // science!|'
remove_punctuation(s)

'data  science'

In [41]:
# loop message column data and creating list
text = []
for i in df['Message']:
    t = remove_punctuation(i)
    text.append(t)

# add cleaned data to df
df['clean_text'] = text
df.head()

,Class,Message,Length,clean_text
0,1,I've been searching for the right words to tha...,196,Ive been searching for the right words to than...
1,0,Free entry in 2 a wkly comp to win FA Cup fina...,155,Free entry in 2 a wkly comp to win FA Cup fina...
2,1,"Nah I don't think he goes to usf, he lives aro...",61,Nah I dont think he goes to usf he lives aroun...
3,1,Even my brother is not like to speak with me. ...,77,Even my brother is not like to speak with me T...
4,1,I HAVE A DATE ON SUNDAY WITH WILL!!!,36,I HAVE A DATE ON SUNDAY WITH WILL


In [42]:
# Spliting the Data
x = df['clean_text']
y = df['Class']

# train test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train.shape

(4453,)

### Using Bag of words techniqe to remove stop words

In [66]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
cv = CountVectorizer(stop_words="english")

In [67]:
# BOW on train data
x_train_cv = cv.fit_transform(x_train)

# BOW on test data
x_test_cv = cv.transform(x_test)

In [68]:
import warnings
warnings.filterwarnings('ignore')
x = cv.get_feature_names_out()

print(x_train_cv.shape)

(4453, 8216)


### Traning the Multinomial Model

In [87]:
# Using Multinomial Model
nb1 = MultinomialNB()
nb1.fit(x_train_cv, y_train)

,"alpha alpha: float or array-like of shape (n_features,), default=1.0Additive (Laplace/Lidstone) smoothing parameter(set alpha=0 and force_alpha=True, for no smoothing).",1.0
,"force_alpha force_alpha: bool, default=TrueIf False and alpha is less than 1e-10, it will set alpha to1e-10. If True, alpha will remain unchanged. This may causenumerical errors if alpha is too close to 0... versionadded:: 1.2.. versionchanged:: 1.4 The default value of `force_alpha` changed to `True`.",True
,"fit_prior fit_prior: bool, default=TrueWhether to learn class prior probabilities or not.If false, a uniform prior will be used.",True
,"class_prior class_prior: array-like of shape (n_classes,), default=NonePrior probabilities of the classes. If specified, the priors are notadjusted according to the data.",None


In [88]:
y_pred1 = nb1.predict(x_test_cv)
y_pred1

array([1, 1, 1, ..., 1, 1, 1], shape=(1114,))

In [71]:
print(classification_report(y_test, y_pred1))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96       145
           1       0.99      0.99      0.99       969

    accuracy                           0.99      1114
   macro avg       0.98      0.98      0.98      1114
weighted avg       0.99      0.99      0.99      1114



In [72]:
print(accuracy_score(y_test, y_pred1))

0.9892280071813285


In [73]:
pd.crosstab(y_test, y_pred1)

col_0,0,1
Class,,
0,139,6
1,6,963


### Conclusion
#### A Multinomial Naive Bayes classifier was trained using Bag-of-Words features with English stopword removal to detect spam messages. The model achieved an accuracy of 98.9% on the test set, with strong precision and recall for both spam and non-spam classes. The confusion matrix indicates very few misclassifications, demonstrating that the model is highly effective at identifying spam messages while maintaining a low false positive rate. This result confirms that traditional NLP techniques combined with Naive Bayes provide a strong and efficient baseline for text classification tasks such as spam detection.

### Training Bernoulli Model

In [85]:
# Using Bernoli Model
bnb1 = BernoulliNB()
bnb1.fit(x_train_cv, y_train)

,"alpha alpha: float or array-like of shape (n_features,), default=1.0Additive (Laplace/Lidstone) smoothing parameter(set alpha=0 and force_alpha=True, for no smoothing).",1.0
,"force_alpha force_alpha: bool, default=TrueIf False and alpha is less than 1e-10, it will set alpha to1e-10. If True, alpha will remain unchanged. This may causenumerical errors if alpha is too close to 0... versionadded:: 1.2.. versionchanged:: 1.4 The default value of `force_alpha` changed to `True`.",True
,"binarize binarize: float or None, default=0.0Threshold for binarizing (mapping to booleans) of sample features.If None, input is presumed to already consist of binary vectors.",0.0
,"fit_prior fit_prior: bool, default=TrueWhether to learn class prior probabilities or not.If false, a uniform prior will be used.",True
,"class_prior class_prior: array-like of shape (n_classes,), default=NonePrior probabilities of the classes. If specified, the priors are notadjusted according to the data.",None


In [86]:
y_pred2 = bnb1.predict(x_test_cv)
y_pred2

array([1, 1, 1, ..., 1, 1, 1], shape=(1114,))

In [76]:
print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

           0       0.99      0.81      0.89       145
           1       0.97      1.00      0.99       969

    accuracy                           0.97      1114
   macro avg       0.98      0.90      0.94      1114
weighted avg       0.97      0.97      0.97      1114



In [77]:
print(accuracy_score(y_test, y_pred2))

0.973967684021544


In [78]:
pd.crosstab(y_test, y_pred2)

col_0,0,1
Class,,
0,117,28
1,1,968


### Conclusion 
#### A Bernoulli Naive Bayes classifier was trained using Bag-of-Words features with English stopword removal to detect spam messages based on the presence or absence of words. The model achieved an accuracy of 97.4% on the test set, with very high recall for spam messages, indicating strong capability in identifying most spam instances. However, the confusion matrix shows a higher number of false positives for non-spam messages compared to the Multinomial Naive Bayes model. This suggests that while Bernoulli Naive Bayes is effective for aggressive spam detection, it is less balanced in preserving legitimate messages. Overall, it provides a reasonable baseline but performs slightly weaker than Multinomial Naive Bayes for this text classification task.

### Using TF-IDF techniqe to remove stop words

In [96]:
tf = TfidfVectorizer(stop_words='english')
x_train_tf = tf.fit_transform(x_train)

x_test_tf = tf.transform(x_test)

### Training Multinomial Model

In [97]:
nb2 = MultinomialNB()
nb2.fit(x_train_tf, y_train)

,"alpha alpha: float or array-like of shape (n_features,), default=1.0Additive (Laplace/Lidstone) smoothing parameter(set alpha=0 and force_alpha=True, for no smoothing).",1.0
,"force_alpha force_alpha: bool, default=TrueIf False and alpha is less than 1e-10, it will set alpha to1e-10. If True, alpha will remain unchanged. This may causenumerical errors if alpha is too close to 0... versionadded:: 1.2.. versionchanged:: 1.4 The default value of `force_alpha` changed to `True`.",True
,"fit_prior fit_prior: bool, default=TrueWhether to learn class prior probabilities or not.If false, a uniform prior will be used.",True
,"class_prior class_prior: array-like of shape (n_classes,), default=NonePrior probabilities of the classes. If specified, the priors are notadjusted according to the data.",None


In [98]:
y_pred3 = nb2.predict(x_test_tf)
y_pred3

array([1, 1, 1, ..., 1, 1, 1], shape=(1114,))

In [99]:
print(accuracy_score(y_pred3, y_test))

0.9730700179533214


In [100]:
print(classification_report(y_pred3, y_test))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88       115
           1       1.00      0.97      0.98       999

    accuracy                           0.97      1114
   macro avg       0.90      0.98      0.93      1114
weighted avg       0.98      0.97      0.97      1114



In [101]:
pd.crosstab(y_pred3, y_test)

Class,0,1
row_0,,
0,115,0
1,30,969


### Conclusion
#### A Multinomial Naive Bayes classifier trained on TF-IDF features achieved strong performance on the spam message dataset, with an overall accuracy of 97.3%. The model demonstrates excellent precision and recall for the spam class, indicating that it can reliably identify spam messages while maintaining a very low false positive rate for non-spam messages. These results confirm that TF-IDF combined with Naive Bayes provides an efficient and effective baseline for text classification tasks such as spam detection.

### Training Bernoulli Model

In [103]:
bnb2 = BernoulliNB()
bnb2.fit(x_train_tf, y_train)

,"alpha alpha: float or array-like of shape (n_features,), default=1.0Additive (Laplace/Lidstone) smoothing parameter(set alpha=0 and force_alpha=True, for no smoothing).",1.0
,"force_alpha force_alpha: bool, default=TrueIf False and alpha is less than 1e-10, it will set alpha to1e-10. If True, alpha will remain unchanged. This may causenumerical errors if alpha is too close to 0... versionadded:: 1.2.. versionchanged:: 1.4 The default value of `force_alpha` changed to `True`.",True
,"binarize binarize: float or None, default=0.0Threshold for binarizing (mapping to booleans) of sample features.If None, input is presumed to already consist of binary vectors.",0.0
,"fit_prior fit_prior: bool, default=TrueWhether to learn class prior probabilities or not.If false, a uniform prior will be used.",True
,"class_prior class_prior: array-like of shape (n_classes,), default=NonePrior probabilities of the classes. If specified, the priors are notadjusted according to the data.",None


In [105]:
y_pred4 = bnb2.predict(x_test_tf)
y_pred4

array([1, 1, 1, ..., 1, 1, 1], shape=(1114,))

In [106]:
print(accuracy_score(y_pred4, y_test))

0.973967684021544


In [107]:
print(classification_report(y_pred4, y_test))

              precision    recall  f1-score   support

           0       0.81      0.99      0.89       118
           1       1.00      0.97      0.99       996

    accuracy                           0.97      1114
   macro avg       0.90      0.98      0.94      1114
weighted avg       0.98      0.97      0.98      1114



In [108]:
pd.crosstab(y_pred4, y_test)

Class,0,1
row_0,,
0,117,1
1,28,968


### Conclusion
#### A Bernoulli Naive Bayes classifier was trained using TF-IDF features to classify spam and non-spam messages. The model achieved an overall accuracy of approximately 97.4% on the test set. The classification report shows strong precision, recall, and F1-scores for both classes, indicating that the model is highly effective at identifying spam messages while maintaining a very low rate of false positives for legitimate messages. The confusion matrix confirms that most messages are correctly classified, with only a small number of misclassifications. This demonstrates that TF-IDF combined with Bernoulli Naive Bayes provides a robust and efficient baseline approach for spam detection tasks.

## Final Conclusion
### In this project, two Naive Bayes variants—Multinomial Naive Bayes and Bernoulli Naive Bayes—were evaluated for SMS spam classification using two feature extraction techniques: Bag-of-Words and TF-IDF, with English stopword removal. Both models demonstrated strong performance, confirming the effectiveness of probabilistic classifiers for text classification tasks.
### Multinomial Naive Bayes with the Bag-of-Words technique achieved higher overall accuracy (≈98.9%) and showed a better balance between precision and recall for both spam and non-spam classes, resulting in fewer false positives and false negatives. Bernoulli Naive Bayes, while achieving very high recall for spam detection, produced a higher number of false positives by incorrectly classifying more legitimate messages as spam.
### Overall, Multinomial Naive Bayes proved to be the more reliable and balanced model for this dataset, making it the preferred choice for deployment in real-world spam detection systems. This comparison highlights that while multiple Naive Bayes variants can perform well, selecting the appropriate model based on data representation (word frequency vs. word presence) is crucial for achieving optimal performance.